# Parselmouth, a tutorial

## Introduction

### Installation

Install Parselmouth by installing the `praat-parselmouth` package from PyPI.

Do not - do or do not? I repeat, do not! - install `parselmouth`, as it is another package. _(Sorry for the confusion.)_

In [ ]:
!pip install praat-parselmouth

And these are some other things you should have installed to run demo:

In [ ]:
!pip install numpy matplotlib tgt

### Let's get started

In [ ]:
import parselmouth

In [ ]:
import numpy as np

In [ ]:
parselmouth.__version__

In [ ]:
parselmouth.PRAAT_VERSION, parselmouth.PRAAT_VERSION_DATE

### What's to come?

- That one example from before
- Fantastic Praat objects and how to plot them
- Accessing "raw" Praat commands
- Running Praat scripts
- Getting annotations from TextGrids
- One language to rule them all: accessing R

**Main goal: how to map a Praat workflow to Parselmouth**

_(in other words: the examples might seem a bit artificial/simplistic, but try to find the principles)_

### Synchronisation point

- Questions/issues/... ?
- https://parselmouth.readthedocs.io/

## That one example from before

In [ ]:
import parselmouth

def extract_info(sound):
    pitch = sound.to_pitch(time_step=0.001, pitch_ceiling=300)
    intensity = sound.to_intensity(75, 0.001, subtract_mean=False)

    print("Here are the results:")
    for i in range(int((sound.tmax - sound.tmin) / 0.01)):
        time = sound.tmin + (i + 1) * 0.01
        p_value = pitch.get_value_at_time(time)
        i_value = intensity.get_value(time)
        print("{:.2f} {:.3f} {:.3f}".format(time, p_value, i_value))

extract_info(parselmouth.Sound("data/the_north_wind_and_the_sun.wav"))

### Let's break that up into pieces

In [ ]:
sound = parselmouth.Sound("data/the_north_wind_and_the_sun.wav")

In [ ]:
sound

In [ ]:
print(sound)

### Seems familiar?

![praat_objects_and_info.png](images/praat_objects_and_info.png)

### Intensity and Pitch

In [ ]:
pitch = sound.to_pitch(time_step=0.001, pitch_ceiling=300)
pitch

In [ ]:
pitch.get_value_at_time(0.5)

![praat_pitch_value.png](images/praat_pitch_get_value.png)

![praat_pitch_value.png](images/praat_pitch_value.png)

In [ ]:
intensity = sound.to_intensity(75, 0.001, subtract_mean=False)
intensity

In [ ]:
intensity.get_value(0.5)

In [ ]:
sound.tmin, sound.tmax

### And that's all Python-Praat interaction

The rest is Python!

In [ ]:
import parselmouth

def extract_info(sound):
    pitch = sound.to_pitch(time_step=0.001, pitch_ceiling=300)
    intensity = sound.to_intensity(75, 0.001, subtract_mean=False)

    print("Here are the results:")
    for i in range(int((sound.tmax - sound.tmin) / 0.01)):
        time = sound.tmin + (i + 1) * 0.01
        p_value = pitch.get_value_at_time(time)
        i_value = intensity.get_value(time)
        print("{:.2f} {:.3f} {:.3f}".format(time, p_value, i_value))

extract_info(parselmouth.Sound("data/the_north_wind_and_the_sun.wav"))

### Synchronisation point

- https://parselmouth.readthedocs.io/en/stable/api_reference.html#parselmouth.Sound
- Change pitch/intensity extraction step to auto
- Extract formant analysis and print first 2 formants

## Fantastic Praat objects and how to plot them

Two goals at once:
1. How to make nice plots and integrate Praat (Parselmouth) into Python plotting libraries
2. How to access the actual data to be plotting **(!)**

In [ ]:
sound = parselmouth.Sound("data/the_north_wind_and_the_sun.wav")  # Or "data/hoover.wav"
sound

In [ ]:
help(sound.get_value)

In [ ]:
sound.n_channels

In [ ]:
sound.get_value(0.5, 1)

In [ ]:
sound.values

In [ ]:
type(sound.values), sound.values.shape

In [ ]:
sound.values[0,:-100:2]

In [ ]:
parselmouth.Sound.__mro__  # Subclass of parselmouth.Matrix

In [ ]:
from IPython.display import Audio

Audio(sound.values, rate=sound.sampling_frequency)

In [ ]:
def audio_player(sound):
    return Audio(sound.values, rate=sound.sampling_frequency)

### Plotting the waveform

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [9, 6]
plt.rcParams['figure.dpi'] = 100

In [ ]:
sound.values[0,:]

In [ ]:
sound.x1 + np.arange(sound.nx) * sound.dx

In [ ]:
sound.xs()

In [ ]:
fig, ax = plt.subplots()
ax.plot(sound.xs(), sound.values[0,:])

In [ ]:
fig, ax = plt.subplots()
ax.plot(sound.xs(), sound.values[0,:], color='pink', linewidth=0.5)
ax.set_xlim(sound.xmin, sound.xmax)
ax.set_ylim(-1, 1)

In [ ]:
sun_sound = sound.extract_part(from_time=0.9, preserve_times=True)
audio_player(sun_sound)

In [ ]:
def plot_waveform(ax, sound, **kwargs):
    max_abs_val = np.max(np.abs(sun_sound.values))
    ax.plot(sun_sound.xs(), sun_sound.values[0,:], **kwargs)
    ax.set_xlim(sound.xmin, sound.xmax)
    ax.set_ylim(-1.2 * max_abs_val, 1.2 * max_abs_val)
    ax.set_xlabel("time (s)")
    ax.set_ylabel("amplitude")

fig, ax = plt.subplots()
plot_waveform(ax, sun_sound, linestyle='--', color='darkolivegreen')

### And now a spectrogram

In [ ]:
spectrogram = sound.to_spectrogram(window_length=0.03)
spectrogram

In [ ]:
help(sound.to_spectrogram)

In [ ]:
isinstance(spectrogram, parselmouth.Matrix), issubclass(parselmouth.Spectrogram, parselmouth.Matrix)

In [ ]:
spectrogram.values

In [ ]:
spectrogram.values.shape, spectrogram.values.size

In [ ]:
spectrogram.values.base

In [ ]:
spectrogram.xs().shape, spectrogram.xs()

In [ ]:
spectrogram.x_grid().shape, spectrogram.x_grid()

In [ ]:
fig, ax = plt.subplots()
ax.pcolormesh(spectrogram.x_grid(), spectrogram.y_grid(), spectrogram.values)

In [ ]:
fig, ax = plt.subplots()
ax.pcolormesh(spectrogram.x_grid(), spectrogram.y_grid(), 10 * np.log10(spectrogram.values))

In [ ]:
def draw_spectrogram(ax, spectrogram, dynamic_range=70, cmap='afmhot'):
    X, Y = spectrogram.x_grid(), spectrogram.y_grid()
    sg_db = 10 * np.log10(spectrogram.values)
    ax.pcolormesh(X, Y, sg_db, vmin=sg_db.max() - dynamic_range, cmap=cmap)
    ax.set_ylim(spectrogram.ymin, spectrogram.ymax)
    ax.set_xlabel("time (s)")
    ax.set_ylabel("frequency (Hz)")

fig, ax = plt.subplots()
draw_spectrogram(ax, spectrogram)

In [ ]:
fig, ax = plt.subplots()
pre_emphasized_sound = sound.copy()
pre_emphasized_sound.pre_emphasize()
draw_spectrogram(ax, pre_emphasized_sound.to_spectrogram(window_length=0.005, maximum_frequency=16000))

### And the pitch contour

In [ ]:
pitch = sound.to_pitch()
pitch

In [ ]:
try:
    pitch.values
except Exception as e:
    import traceback
    traceback.print_exc()

In [ ]:
pitch.get_frame(20)

In [ ]:
pitch.get_frame(20).candidates

In [ ]:
pitch.get_frame(20).candidates[0].frequency

![praat_pitch_candidates.png](images/praat_pitch_candidates.png)

In [ ]:
pitch.selected_array

In [ ]:
def draw_pitch(ax, pitch):
    # Extract selected pitch contour, and replace unvoiced samples by NaN to not plot
    pitch_values = pitch.selected_array['frequency']
    pitch_values[pitch_values==0] = np.nan
    ax.plot(pitch.xs(), pitch_values, 'o', markersize=5, color='w')
    ax.plot(pitch.xs(), pitch_values, 'o', markersize=2)
    ax.grid(False)
    ax.set_ylim(0, pitch.ceiling)
    ax.set_ylabel("fundamental frequency (Hz)")

In [ ]:
fig, ax = plt.subplots()
draw_spectrogram(ax, spectrogram)
ax_twin = ax.twinx()
draw_pitch(ax_twin, pitch)
ax.set_xlim(sound.xmin, sound.xmax)

In [ ]:
hoover_sound = parselmouth.Sound("data/hoover.wav")
audio_player(hoover_sound)

In [ ]:
fig, ax = plt.subplots()
draw_spectrogram(ax, hoover_sound.to_spectrogram(window_length=0.03), dynamic_range=50, cmap='inferno')
ax.set_xlim(hoover_sound.xmin, hoover_sound.xmax)

### Synchronisation point

- https://parselmouth.readthedocs.io/en/stable/examples/plotting.html
- Play around with styles and colors and colormaps
- Plot a formant track

## Accessing "raw" Praat commands

Creating a Python interface is quite a bit of work. See source ;-)

What to do when no Python class exists?

In [ ]:
manipulation = parselmouth.praat.call(sound, "To Manipulation", 0.01, 75, 600)
manipulation

In [ ]:
type(manipulation), type(sound)

In [ ]:
parselmouth.Sound.__mro__

In [ ]:
help(parselmouth.Data)

In [ ]:
manipulation.class_name

`parselmouth.Data` is the base class of all Praat objects (actually `parselmouth.Thing`; it's complicated).

When the full type is not exposed in Parselmouth, you get a `parselmouth.Data`.

https://parselmouth.readthedocs.io/en/stable/api_reference.html#parselmouth.praat.call

In [ ]:
help(parselmouth.praat.call)

In [ ]:
audio_player(sound)

In [ ]:
from parselmouth.praat import call

manipulation = call(sound, "To Manipulation", 0.01, 75, 600)

pitch_tier = call(manipulation, "Extract pitch tier")

call(pitch_tier, "Multiply frequencies", sound.xmin, sound.xmax, 2)

call([pitch_tier, manipulation], "Replace pitch tier")
sound_octave_up = call(manipulation, "Get resynthesis (overlap-add)")
sound_octave_up

In [ ]:
audio_player(sound_octave_up)

In [ ]:
try:
    call(sound, "View & Edit")
except Exception as e:
    import traceback
    traceback.print_exc()

Not supported:
- Interactive "View & Edit" and menus: see main menu
- Drawing: hopefully future interaction with matplotlib

### Synchronization point

- https://parselmouth.readthedocs.io/en/stable/api_reference.html#parselmouth.praat.call
- https://parselmouth.readthedocs.io/en/stable/examples/pitch_manipulation.html
- Is your favorite/most-used piece of Praat functionality available? Can you call it?

## Running Praat scripts

Running praat scripts is very similar to running a single Praat command. A run has two kinds of inputs:
- selected Praat objects
- argument values for Praat `form` parameters

Contrary to actions, a script can have three kinds of outputs:
- Praat objects
- printed output (optional)
- variables (optional)

Same rule as for actions: no interactive stuff. We're programming, after all, not running the GUI.

https://parselmouth.readthedocs.io/en/stable/api_reference.html#parselmouth.praat.run

In [ ]:
praat_script = """\
form Increase the pitch by 1 octave
    positive Pitch_floor 75
    positive Pitch_ceiling 600
endform

sound = selected()

manipulation = To Manipulation: 0.01, pitch_floor, pitch_ceiling

pitch_tier = Extract pitch tier

select sound
xmin = Get start time
xmax = Get end time

appendInfoLine: "xmin = ", xmin
appendInfoLine: "xmax = ", xmax

select pitch_tier
Multiply frequencies: xmin, xmax, 2

select pitch_tier
plus manipulation
Replace pitch tier

select manipulation
Get resynthesis (overlap-add)
"""

In [ ]:
parselmouth.praat.run(sound, praat_script, 75, 600)

In [ ]:
praat_script_bis = praat_script + """\
select manipulation
plus pitch_tier
plus sound
"""

In [ ]:
parselmouth.praat.run(sound, praat_script, 75, 600, capture_output=True)

In [ ]:
objects, output = parselmouth.praat.run(sound, praat_script, 75, 600, capture_output=True)
print(output)

In [ ]:
objects, variables = parselmouth.praat.run(sound, praat_script, 75, 600, return_variables=True)
variables

### Re-using "legacy" Praat code

It is reasonably simple to reuse an existing piece of Praat code. Often, a few changes at the start and end are enough.

For example, let's take the existing "Praat Script Syllable Nuclei v2" script. (De Jong, N. H., & Wempe, T. (2009). Praat script to detect syllable nuclei and measure speech rate automatically. _Behavior Research Methods, 41_, 385–390.)

Compare `data/syllable_nuclei_orig.praat` and `data/syllable_nuclei.praat`.

![syllable_nuclei_diff_1.png](images/syllable_nuclei_diff_1.png)

![syllable_nuclei_diff_2.png](images/syllable_nuclei_diff_2.png)

In [ ]:
def extract_syllable_intervals(file_name):
    print("Extracting syllable intervals from '{}'...".format(file_name))

    objects = parselmouth.praat.run_file("data/syllable_nuclei.praat", -25, 2, 0.3, file_name)
    text_grid = objects[1]
    n = parselmouth.praat.call(text_grid, "Get number of points", 1)
    syllable_nuclei = [parselmouth.praat.call(text_grid, "Get time of point", 1, i + 1) for i in range(n)]

    syllable_intervals = np.diff(syllable_nuclei)
    return syllable_nuclei, syllable_intervals

In [ ]:
nuclei, intervals = extract_syllable_intervals("the_north_wind_and_the_sun.wav")

In [ ]:
nuclei

In [ ]:
intervals

In [ ]:
fig, ax = plt.subplots()
draw_spectrogram(ax, sound.to_spectrogram(window_length=0.03))
ax.set_xlim(sound.xmin, sound.xmax)
for nucleus in nuclei:
    ax.axvline(nucleus, color='green', linewidth=3)

### Synchronization point

- https://parselmouth.readthedocs.io/en/stable/api_reference.html#parselmouth.praat.run
- https://parselmouth.readthedocs.io/en/stable/api_reference.html#parselmouth.praat.run_script
- Run a script from http://www.praatvocaltoolkit.com/
- Any own Praat scripts to run?

## Getting annotations from TextGrids

### TextGridTools

There is no full support for TextGrids in Parselmouth (yet) through the "Pythonic" API. However, `tgt` (TextGridTools) is a small Python library that perfectly works together with Python and Parselmouth.

In [ ]:
import tgt

text_grid = tgt.io.read_textgrid("data/the_north_wind_and_the_sun.TextGrid")
text_grid

In [ ]:
text_grid.get_tier_names()

In [ ]:
text_grid.get_tier_by_name('syllable nuclei')

In [ ]:
sound = parselmouth.Sound("data/the_north_wind_and_the_sun.wav")
intensity = sound.to_intensity()

In [ ]:
syllable_nuclei_tier = text_grid.get_tier_by_name('syllable nuclei')
for point in syllable_nuclei_tier.points:
    t = point.time
    t_intensity = intensity.get_value(t)
    print(f"{t:.4f}: {t_intensity:.2f} ({point.text})")

In [ ]:
a_parselmouth_text_grid = parselmouth.read("data/the_north_wind_and_the_sun.TextGrid")
a_parselmouth_text_grid

In [ ]:
a_tgt_text_grid = a_parselmouth_text_grid.to_tgt()
a_tgt_text_grid

In [ ]:
a_tgt_text_grid.get_tier_names()

In [ ]:
parselmouth.TextGrid.from_tgt(a_tgt_text_grid)

In [ ]:
call(parselmouth.TextGrid.from_tgt(a_tgt_text_grid), "Get tier name", 2)

### Synchronization point

- https://textgridtools.readthedocs.io/
- Read in some annotations and plot them on top of a spectrogram?

## One language to rule them all: accessing R

In [ ]:
!pip install rpy2

In [ ]:
pitch_track = pitch.selected_array['frequency']
pitch_track[:20]

In [ ]:
from rpy2.robjects import FloatVector
from rpy2.robjects.packages import importr

r_stats = importr('stats')
r_stats

In [ ]:
result = r_stats.wilcox_test(FloatVector(pitch_track[:len(pitch) // 2]),
                             FloatVector(pitch_track[len(pitch) // 2:]))
result

In [ ]:
result.rx2('p.value')[0]

### Synchronization point

- https://rpy2.github.io/
- https://rpy2.github.io/doc/latest/html/index.html
- Run another stats test from R on your own data

## Extra: your name in a spectrogram

In [ ]:
!pip install librosa Pillow

In [ ]:
import librosa
import numpy as np
import parselmouth
from PIL import Image, ImageDraw, ImageFont

WORD = "Parselmouth"
SIZE = 300
MARGIN = 50
FONT = "DejaVuSans-Bold.ttf"  # "Comic_Sans_MS.ttf"

TIME_STEP = 0.0025
MAX_FREQUENCY = 5000
SAMPLING_FREQUENCY = 44100

font = ImageFont.truetype(FONT, SIZE)
size = font.getbbox(WORD, anchor='lt')[2:]
image = Image.new('L', (size[0] + 2 * MARGIN, size[1] + 2 * MARGIN))
draw = ImageDraw.Draw(image)
draw.text((MARGIN, MARGIN), WORD, fill=255, font=font, anchor='lt')

n, m = image.size
matrix = parselmouth.praat.call("Create Matrix", "", 0, TIME_STEP * n, n, TIME_STEP, TIME_STEP / 2, 0, MAX_FREQUENCY, m, MAX_FREQUENCY / m, MAX_FREQUENCY / m / 2, "0")
matrix.values = np.array(image)[::-1,:] / 255
spectrogram = parselmouth.praat.call(matrix, "To Spectrogram")
logospectrogram = spectrogram.to_sound(SAMPLING_FREQUENCY)
logospectrogram.scale_peak()
logospectrogram.save("logospectrogram.wav", 'WAV')

librosa_spectrogram = np.zeros((int(m / 5000 * SAMPLING_FREQUENCY / 2), n))
librosa_spectrogram[:m,:n] = np.array(image)[::-1,:] / 255
logospectrogram_librosa = parselmouth.Sound(librosa.griffinlim(librosa_spectrogram, hop_length=int(TIME_STEP * SAMPLING_FREQUENCY)))
logospectrogram_librosa.scale_peak()
logospectrogram_librosa.save("logospectrogram_librosa.wav", 'WAV')

In [ ]:
audio_player(logospectrogram)

In [ ]:
fig, ax = plt.subplots()
draw_spectrogram(ax, logospectrogram.to_spectrogram(window_length=0.03))
ax.set_xlim(logospectrogram.xmin, logospectrogram.xmax)

In [ ]:
audio_player(logospectrogram_librosa)

In [ ]:
fig, ax = plt.subplots()
draw_spectrogram(ax, logospectrogram_librosa.to_spectrogram(window_length=0.03))
ax.set_xlim(logospectrogram_librosa.xmin, logospectrogram_librosa.xmax)

## Extra: interactive application demo

## That's all I have (in this tutorial)

**Questions or requests?**